In [2]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

### Data loaders
We wrap the data loaders in their own function and pass a global data directory. This way we can share a data directory between different trials.

In [4]:
def load_data(data_dir="./data"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # 3 channels, mean and standard deviation
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

### Configurable neural network

In [7]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Full training function

In [8]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

### Test set accuracy

In [9]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [10]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

170499072it [00:41, 4098629.50it/s]                               


Extracting c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\[3]Pytorch\data\cifar-10-python.tar.gz to c:\Users\HYPC300\OneDrive - 한양대학교\GitHub\Deeplearning-Autoencoder-DOA\[3]Pytorch\data
Files already downloaded and verified


2022-02-18 20:37:18,276	WARNING experiment.py:255 -- No name detected on trainable. Using DEFAULT.
2022-02-18 20:37:18,278	INFO registry.py:69 -- Detected unknown callable for trainable. Converting to class.
2022-02-18 20:37:18,658	ERROR syncer.py:111 -- Log sync requires rsync to be installed.


== Status ==
Current time: 2022-02-18 20:37:18 (running for 00:00:00.28)
Memory usage on this node: 20.6/31.8 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/12 CPUs, 0/1 GPUs, 0.0/7.24 GiB heap, 0.0/3.62 GiB objects
Result logdir: C:\Users\HYPC300\ray_results\DEFAULT_2022-02-18_20-37-18
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-----------------+--------------+------+------+-------------+
| Trial name          | status   | loc             |   batch_size |   l1 |   l2 |          lr |
|---------------------+----------+-----------------+--------------+------+------+-------------|
| DEFAULT_20a51_00000 | RUNNING  | 127.0.0.1:44984 |            2 |    8 |    4 | 0.00250225  |
| DEFAULT_20a51_00001 | PENDING  |                 |            4 |    4 |  256 | 0.0121433   |
| DEFAULT_20a51_00002 | PENDING  |                 |            8 |   32 |    8 | 0.

2022-02-18 20:44:52,162	WARNING tune.py:592 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


== Status ==
Current time: 2022-02-18 20:44:52 (running for 00:07:33.77)
Memory usage on this node: 22.3/31.8 GiB
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: -1.552187786269188 | Iter 1.000: -1.8936227770268916
Resources requested: 10.0/12 CPUs, 0/1 GPUs, 0.0/7.24 GiB heap, 0.0/3.62 GiB objects
Result logdir: C:\Users\HYPC300\ray_results\DEFAULT_2022-02-18_20-37-18
Number of trials: 10/10 (5 RUNNING, 5 TERMINATED)
+---------------------+------------+-----------------+--------------+------+------+-------------+---------+------------+----------------------+
| Trial name          | status     | loc             |   batch_size |   l1 |   l2 |          lr |    loss |   accuracy |   training_iteration |
|---------------------+------------+-----------------+--------------+------+------+-------------+---------+------------+----------------------|
| DEFAULT_20a51_00003 | RUNNING    | 127.0.0.1:38900 |           16 |   16 |   16 | 0.000951491 | 1

 pid=46640) Traceback (most recent call last):
 pid=46640)   File "<string>", line 1, in <module>
 pid=46640)   File "C:\Users\HYPC300\anaconda3\envs\basic_NN_study\lib\multiprocessing\spawn.py", line 116, in spawn_main
 pid=46640)     exitcode = _main(fd, parent_sentinel)
 pid=46640)   File "C:\Users\HYPC300\anaconda3\envs\basic_NN_study\lib\multiprocessing\spawn.py", line 124, in _main
 pid=46640)     preparation_data = reduction.pickle.load(from_parent)
 pid=46640) EOFError: Ran out of input
 pid=38436) Traceback (most recent call last):
 pid=38436)   File "<string>", line 1, in <module>
 pid=38436)   File "C:\Users\HYPC300\anaconda3\envs\basic_NN_study\lib\multiprocessing\spawn.py", line 116, in spawn_main
 pid=38436)     exitcode = _main(fd, parent_sentinel)
 pid=38436)   File "C:\Users\HYPC300\anaconda3\envs\basic_NN_study\lib\multiprocessing\spawn.py", line 124, in _main
 pid=38436)     preparation_data = reduction.pickle.load(from_parent)
 pid=38436) EOFError: Ran out of input


Best trial config: {'l1': 64, 'l2': 4, 'lr': 0.0014556616076764708, 'batch_size': 8}
Best trial final validation loss: 1.383349322795868
Best trial final validation accuracy: 0.5001
Files already downloaded and verified


 pid=44324) Traceback (most recent call last):
 pid=44324)   File "<string>", line 1, in <module>
 pid=44324)   File "C:\Users\HYPC300\anaconda3\envs\basic_NN_study\lib\multiprocessing\spawn.py", line 116, in spawn_main
 pid=44324)     exitcode = _main(fd, parent_sentinel)
 pid=44324)   File "C:\Users\HYPC300\anaconda3\envs\basic_NN_study\lib\multiprocessing\spawn.py", line 126, in _main
 pid=44324)     self = reduction.pickle.load(from_parent)
 pid=44324) _pickle.UnpicklingError: pickle data was truncated
 pid=38900) Traceback (most recent call last):
 pid=38900)   File "<string>", line 1, in <module>
 pid=38900)   File "C:\Users\HYPC300\anaconda3\envs\basic_NN_study\lib\multiprocessing\spawn.py", line 116, in spawn_main
 pid=38900)     exitcode = _main(fd, parent_sentinel)
 pid=38900)   File "C:\Users\HYPC300\anaconda3\envs\basic_NN_study\lib\multiprocessing\spawn.py", line 126, in _main
 pid=38900)     self = reduction.pickle.load(from_parent)
 pid=38900) _pickle.UnpicklingError: pi

Files already downloaded and verified
Best trial test set accuracy: 0.5014
